<a href="https://colab.research.google.com/github/alen768/Data_Collection_with_API/blob/main/Data_Collection(withAPI).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
import base64

In [12]:
CLIENT_ID = 'CLIENT ID OF YOURS'
CLIENT_SECRET = 'CLIENT SECRET OF YOURS'

In [4]:
#base64 encode the client id and client secret
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

In [5]:
#request access token
token_url = "https://accounts.spotify.com/api/token"

headers = {
    'Authorization' : f'Basic {client_credentials_base64.decode()}'
}

data = {
    'grant_type': 'client_credentials'
}

response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
  access_token = response.json()['access_token']
  print("Access token obtained successfully")
else:
  print("Error obtaining access token")
  exit()

Access token obtained successfully


In [6]:
pip install spotipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 6.2 MB/s eta 0:00:00


In [8]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

def get_trending_playlist_data(playlist_id, access_token):

  #set up spotipy with access token
  sp = spotipy.Spotify(auth=access_token)

  #getting tracks from the playlist
  playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id,name,artists,album(id,name)))')

  #extracting relevant info and storing it in a list of dictionaries
  music_data = []
  for track_info in playlist_tracks['items']:
    track = track_info['track']
    track_name = track['name']
    artists = ','.join([artist['name'] for artist in track['artists']])
    album_name = track['album']['name']
    album_id = track['album']['id']
    track_id = track['id']

    #getting audio feautures for the track
    audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

    #getting release date of albums
    try:
      album_info = sp.album(album_id) if album_id != 'Not available' else None
      release_date = album_info['release_date'] if album_info else None
    except:
      release_date = None

    #getting popularity of the track
    try:
      track_info = sp.track(track_id) if track_id != 'Not available' else None
      popularity = track_info['popularity'] if track_info else None
    except:
      popularity = None

    #adding additional track information to track data
    track_data = {
        'Track Name' : track_name,
        'Artists' : artists,
        'Album Name' : album_name,
        'Album ID' : album_id,
        'Track ID' : track_id,
        'Popularity' : popularity,
        'Release Date' : release_date,
        'Duration(ms)' : audio_features['duration_ms'] if audio_features else None,
        'Explicit' : track_info.get('explicit',None),
        'External URLs' : track_info.get('external_urls', {}).get('spotify', None),
        'Danceability' : audio_features['danceability'] if audio_features else None,
        'Energy' : audio_features['energy'] if audio_features else None,
        'Key' : audio_features['key'] if audio_features else None,
        'Loudness' : audio_features['loudness'] if audio_features else None,
        'Mode' : audio_features['mode'] if audio_features else None,
        'Speechiness' : audio_features['speechiness'] if audio_features else None,
        'Acousticness' : audio_features['acousticness'] if audio_features else None,
        'Instrumentalness' : audio_features['instrumentalness'] if audio_features else None,
        'Liveness' : audio_features['liveness'] if audio_features else None,
        'Valence' : audio_features['valence'] if audio_features else None,
        'Tempo' : audio_features['tempo'] if audio_features else None,
    }

    music_data.append(track_data)

  #creating a pandas DataFrame from the list of dictionary
    df = pd.DataFrame(music_data)
    return df

In [11]:
#taking any playlist id from spotify
playlist_id = '37i9dQZF1E39c9drv2MTb2'

#calling the function and storing it in a dataframe
music_df = get_trending_playlist_data(playlist_id,access_token)
print(music_df)

  Track Name              Artists  \
0  Kadharalz  Anirudh Ravichander   

                                      Album Name                Album ID  \
0  Indian 2 (Original Motion Picture Soundtrack)  2odSkyvlROPRtPYsr2lE0l   

                 Track ID  Popularity Release Date  Duration(ms)  Explicit  \
0  77OQsnAirm7Ke8EGgBeOxz          62   2024-06-01        234023     False   

                                       External URLs  ...  Energy  Key  \
0  https://open.spotify.com/track/77OQsnAirm7Ke8E...  ...   0.902    1   

   Loudness  Mode  Speechiness  Acousticness  Instrumentalness  Liveness  \
0    -5.411     1        0.298        0.0638           0.00466     0.539   

   Valence    Tempo  
0    0.735  160.025  

[1 rows x 21 columns]


In [ ]:
#saving as CSV file if needed
#music_df.to_csv('musicdata.csv')